# <p style="text-align: center;, font-style: strong;">Partie 2 : MNIST with Multiple Layer Perceptron (MLP)</p>

### <p style="text-align: center;">(Almond 0.8.1, Scala 2.12.8)</p>


## Dependencies

In [ ]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("linux-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [ ]:
import java.nio.file.Paths

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.Indexer._
import org.platanios.tensorflow.api.core.client.Session
import org.platanios.tensorflow.data.image.MNISTLoader

import org.platanios.tensorflow.api.learn.layers.{ Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean }
import org.platanios.tensorflow.api.learn.{ Model, StopCriteria }
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator


## Display MNIST Dataset

In [ ]:
{{
def displayNumberMNIST(nb: Int) {
    val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
    val images = dataset.trainImages
    val imagesToDisplay = images.slice(0 :: nb, ::, ::)
    for (index <- 0 until nb) {
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imagesToDisplay(index).reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}
displayNumberMNIST(20)
}}

In [ ]:
val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
val trainImages = tf.data.datasetFromTensorSlices(dataset.trainImages.toFloat)

val trainLabels = tf.data.datasetFromTensorSlices(dataset.trainLabels.toLong)
val trainData =
  trainImages.zip(trainLabels)
      .repeat()
      .shuffle(10000)
      .batch(256)
      .prefetch(10)

// Create the MLP model.
val input = Input(FLOAT32, Shape(-1, 28, 28))
val trainInput = Input(INT64, Shape(-1))
val layer = Flatten[Float]("Input/Flatten") >>
    Linear[Float]("Layer_0", 256) >> ReLU[Float]("Layer_0/Activation") >>
    Linear[Float]("Layer_1", 256) >> ReLU[Float]("Layer_1/Activation") >>
    Linear[Float]("OutputLayer", 10)
val loss = SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss") >>
    Mean("Loss/Mean")
val optimizer = tf.train.Adam()
val model = Model.simpleSupervised(input, trainInput, layer, loss, optimizer)

// Create an estimator and train the model.
val estimator = InMemoryEstimator(model)
estimator.train(() => trainData, StopCriteria(maxSteps = Some(5000)))


In [ ]:
def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
    val predictions = estimator.infer(() => images.toFloat)
    predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}

println(s"Train accuracy = ${accuracy(dataset.trainImages, dataset.trainLabels)}")
println(s"Test accuracy = ${accuracy(dataset.testImages, dataset.testLabels)}")

## Test results

In [ ]:
val images = dataset.testImages

def inferOnSelectedImage(indexes: Seq[Int], images: Tensor[UByte]) {
    indexes.foreach { index => 
        val imageToInfer = images.slice(index, ::, ::).reshape(Shape(1, 28, 28))
        val predictions = estimator.infer(() => imageToInfer.toFloat)
        println(s"Label infered: ${predictions.argmax(1).scalar}")
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imageToInfer.reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}

inferOnSelectedImage((30 to 40), images)
